# Adaptive QNN with 10 Qubits
## Feature Selection + Encoding Optimization

**Experiment Overview:**
- Tests adaptive controller with 10 qubits
- Compares 4 feature selection methods × 2 encodings = 8 configurations
- 30 epochs of training

In [1]:
# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running on Google Colab")

    # Check GPU
    !nvidia-smi

    # Install packages
    !pip install -q pennylane pennylane-lightning-gpu
    !pip install -q scikit-learn matplotlib seaborn tqdm
except:
    IN_COLAB = False
    print("Running locally")

print("\n✓ Setup complete!")

Running on Google Colab
Wed Oct 15 19:09:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------

In [2]:
import numpy as np
import pennylane as qml
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
from collections import defaultdict
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✓ Imports complete")

✓ Imports complete


## Configuration

In [3]:
class Config:
    """Experiment configuration"""
    N_QUBITS = 10
    N_LAYERS = 2
    LEARNING_RATE = 0.01
    N_EPOCHS = 30
    BATCH_SIZE = 16

    # Adaptive strategy
    STRATEGY = 'ucb'  # Options: 'ucb', 'epsilon_greedy', 'round_robin'
    UCB_C = 2.0
    EPSILON = 0.2

    # Dataset
    DATASET = 'wine'  # Options: 'iris', 'wine', 'breast_cancer'
    TEST_SIZE = 0.3
    RANDOM_STATE = 42

    # Device - try GPU first
    try:
        test_dev = qml.device('lightning.gpu', wires=2)
        DEVICE_NAME = 'lightning.gpu'
        print("✓ GPU acceleration available")
    except:
        DEVICE_NAME = 'default.qubit'
        print("Using CPU (GPU not available)")

    # Output
    VERBOSE = True
    PLOT_RESULTS = True
    SAVE_RESULTS = True
    OUTPUT_DIR = Path('results/10_qubits')

config = Config()
config.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"\nConfiguration:")
print(f"  Qubits: {config.N_QUBITS}")
print(f"  Layers: {config.N_LAYERS}")
print(f"  Epochs: {config.N_EPOCHS}")
print(f"  Strategy: {config.STRATEGY}")
print(f"  Dataset: {config.DATASET}")
print(f"  Device: {config.DEVICE_NAME}")

✓ GPU acceleration available

Configuration:
  Qubits: 10
  Layers: 2
  Epochs: 30
  Strategy: ucb
  Dataset: wine
  Device: lightning.gpu


## Data Loading

In [4]:
class DataLoader:
    """Load and preprocess datasets"""

    @staticmethod
    def load_dataset(name='wine'):
        """Load specified dataset"""
        datasets = {
            'iris': load_iris,
            'wine': load_wine,
            'breast_cancer': load_breast_cancer
        }

        if name not in datasets:
            raise ValueError(f"Dataset {name} not recognized")

        data = datasets[name]()
        X, y = data.data, data.target

        # Binary classification (first 2 classes)
        mask = y < 2
        X, y = X[mask], y[mask]

        return X, y, data.feature_names

    @staticmethod
    def preprocess(X_train, X_test, y_train, y_test):
        """Standardize features"""
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        return X_train, X_test, y_train, y_test

print("✓ DataLoader defined")

✓ DataLoader defined


## Feature Selection Methods

In [5]:
class FeatureSelector:
    """Feature selection methods"""

    @staticmethod
    def pca(X_train, X_test, n_components):
        """PCA dimensionality reduction"""
        pca = PCA(n_components=n_components)
        X_train_red = pca.fit_transform(X_train)
        X_test_red = pca.transform(X_test)
        return X_train_red, X_test_red

    @staticmethod
    def correlation(X_train, X_test, y_train, n_features):
        """Select features by correlation with target"""
        correlations = np.array([np.corrcoef(X_train[:, i], y_train)[0, 1]
                                for i in range(X_train.shape[1])])
        top_indices = np.argsort(np.abs(correlations))[-n_features:]
        return X_train[:, top_indices], X_test[:, top_indices]

    @staticmethod
    def mutual_info(X_train, X_test, y_train, n_features):
        """Select features by mutual information"""
        mi_scores = mutual_info_classif(X_train, y_train, random_state=42)
        top_indices = np.argsort(mi_scores)[-n_features:]
        return X_train[:, top_indices], X_test[:, top_indices]

    @staticmethod
    def random_selection(X_train, X_test, n_features):
        """Random feature selection (baseline)"""
        np.random.seed(42)
        indices = np.random.choice(X_train.shape[1], n_features, replace=False)
        return X_train[:, indices], X_test[:, indices]

print("✓ FeatureSelector defined")

✓ FeatureSelector defined


## Quantum Encoding

In [6]:
class QuantumEncoder:
    """Quantum data encoding methods"""

    @staticmethod
    def angle_encoding(features, wires):
        """Encode data as rotation angles"""
        for i, wire in enumerate(wires):
            if i < len(features):
                qml.RY(features[i], wires=wire)

    @staticmethod
    def amplitude_encoding(features, wires):
        """Encode data as quantum state amplitudes"""
        n_amplitudes = 2 ** len(wires)
        padded = np.zeros(n_amplitudes)
        padded[:len(features)] = features
        normalized = padded / np.linalg.norm(padded) if np.linalg.norm(padded) > 0 else padded
        qml.AmplitudeEmbedding(normalized, wires=wires, normalize=True)

print("✓ QuantumEncoder defined")

✓ QuantumEncoder defined


## Quantum Neural Network

In [19]:
from autograd import numpy as anp

class QuantumNeuralNetwork:
    """Parameterized quantum circuit for classification"""

    def __init__(self, n_qubits, n_layers, encoding_method='angle'):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.encoding_method = encoding_method

        # Initialize quantum device
        try:
            self.dev = qml.device(config.DEVICE_NAME, wires=n_qubits)
        except:
            print(f"Warning: {config.DEVICE_NAME} not available, using default.qubit")
            self.dev = qml.device('default.qubit', wires=n_qubits)

        # Initialize parameters as numpy array
        self.n_params = n_layers * n_qubits * 3  # 3 rotations per qubit per layer
        self.params = np.random.randn(self.n_params) * 0.1

        # Create QNode WITH interface='autograd' - CRITICAL FIX
        self.qnode = qml.QNode(self._circuit, self.dev, interface='autograd')

    def _circuit(self, features, params):
        """Quantum circuit architecture"""
        wires = range(self.n_qubits)

        # Data encoding
        if self.encoding_method == 'angle':
            QuantumEncoder.angle_encoding(features, wires)
        elif self.encoding_method == 'amplitude':
            QuantumEncoder.amplitude_encoding(features, wires)
        else:
            raise ValueError(f"Unknown encoding method: {self.encoding_method}")

        # Parameterized layers
        for layer in range(self.n_layers):
            # Single-qubit rotations
            for i in range(self.n_qubits):
                idx = layer * self.n_qubits * 3 + i * 3
                qml.RX(params[idx], wires=i)
                qml.RY(params[idx + 1], wires=i)
                qml.RZ(params[idx + 2], wires=i)

            # Entangling layer (circular connectivity)
            for i in range(self.n_qubits):
                qml.CNOT(wires=[i, (i + 1) % self.n_qubits])

        # Measurement
        return qml.expval(qml.PauliZ(0))

    def predict_single(self, features):
        """Predict for a single sample"""
        output = self.qnode(features, self.params)
        return 1 if output > 0 else 0

    def predict_batch(self, X):
        """Predict batch of samples"""
        return np.array([self.predict_single(x) for x in X])

    def train_step(self, X_batch, y_batch, learning_rate):
        """Single training step with gradient descent"""
        def loss_fn(params):
            # Use autograd numpy for operations inside gradient computation
            predictions = anp.array([self.qnode(x, params) for x in X_batch])
            targets = 2 * y_batch - 1
            return anp.mean((predictions - targets) ** 2)

        # Compute gradients - argnum=0 specifies first argument
        grad_fn = qml.grad(loss_fn, argnum=0)
        gradients = grad_fn(self.params)

        # Update parameters (regular numpy is fine here)
        self.params = self.params - learning_rate * np.array(gradients)

        return float(loss_fn(self.params))

print("✓ QuantumNeuralNetwork defined")

✓ QuantumNeuralNetwork defined


## Adaptive Controller

In [20]:
class AdaptiveController:
    """Adaptive configuration selection"""

    def __init__(self, configurations, strategy='ucb'):
        self.configurations = configurations
        self.strategy = strategy

        # Tracking
        self.rewards = {cfg: [] for cfg in configurations}
        self.selection_counts = {cfg: 0 for cfg in configurations}
        self.total_selections = 0

        # Strategy parameters
        self.ucb_c = config.UCB_C
        self.epsilon = config.EPSILON

    def select_configuration(self):
        """Select next configuration to try"""
        if self.strategy == 'round_robin':
            idx = self.total_selections % len(self.configurations)
            return self.configurations[idx]

        elif self.strategy == 'epsilon_greedy':
            if np.random.random() < self.epsilon or self.total_selections < len(self.configurations):
                # Explore
                return np.random.choice(self.configurations)
            else:
                # Exploit
                return self.get_best_configuration()

        elif self.strategy == 'ucb':
            # Ensure each config tried at least once
            if self.total_selections < len(self.configurations):
                return self.configurations[self.total_selections]

            # UCB selection
            ucb_values = {}
            for cfg in self.configurations:
                if len(self.rewards[cfg]) == 0:
                    ucb_values[cfg] = float('inf')
                else:
                    mean_reward = np.mean(self.rewards[cfg])
                    exploration = self.ucb_c * np.sqrt(
                        np.log(self.total_selections) / self.selection_counts[cfg]
                    )
                    ucb_values[cfg] = mean_reward + exploration

            return max(ucb_values, key=ucb_values.get)

    def update(self, configuration, reward):
        """Update with new reward"""
        self.rewards[configuration].append(reward)
        self.selection_counts[configuration] += 1
        self.total_selections += 1

    def get_best_configuration(self):
        """Get configuration with highest mean reward"""
        mean_rewards = {
            cfg: np.mean(rewards) if rewards else 0
            for cfg, rewards in self.rewards.items()
        }
        return max(mean_rewards, key=mean_rewards.get)

    def get_statistics(self):
        """Get performance statistics"""
        stats = []
        for cfg in self.configurations:
            if self.rewards[cfg]:
                stats.append({
                    'configuration': cfg,
                    'mean_reward': np.mean(self.rewards[cfg]),
                    'std_reward': np.std(self.rewards[cfg]),
                    'count': len(self.rewards[cfg])
                })
        return sorted(stats, key=lambda x: x['mean_reward'], reverse=True)

print("✓ AdaptiveController defined")

✓ AdaptiveController defined


## Main Training Pipeline

In [21]:
class AdaptiveQNNTrainer:
    """Main training pipeline"""

    def __init__(self):
        # Load data
        print("Loading dataset...")
        X, y, feature_names = DataLoader.load_dataset(config.DATASET)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=config.TEST_SIZE, random_state=config.RANDOM_STATE
        )
        X_train, X_test, y_train, y_test = DataLoader.preprocess(
            X_train, X_test, y_train, y_test
        )

        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

        print(f"Dataset: {config.DATASET}")
        print(f"Training: {len(X_train)}, Test: {len(X_test)}")
        print(f"Features: {X_train.shape[1]} → {config.N_QUBITS}")

        # Define configurations
        feature_methods = ['PCA', 'Correlation', 'MutualInfo', 'Random']
        encoding_methods = ['angle', 'amplitude']

        self.configurations = [
            f"{fm}+{em}" for fm in feature_methods for em in encoding_methods
        ]

        # Initialize controller
        self.controller = AdaptiveController(self.configurations, config.STRATEGY)

        # Tracking
        self.training_history = {
            'train_acc': [],
            'val_acc': [],
            'config': [],
            'epoch_time': []
        }

        self.start_time = time.time()

    def apply_feature_selection(self, method):
        """Apply feature selection"""
        if method == 'PCA':
            return FeatureSelector.pca(self.X_train, self.X_test, config.N_QUBITS)
        elif method == 'Correlation':
            return FeatureSelector.correlation(
                self.X_train, self.X_test, self.y_train, config.N_QUBITS
            )
        elif method == 'MutualInfo':
            return FeatureSelector.mutual_info(
                self.X_train, self.X_test, self.y_train, config.N_QUBITS
            )
        elif method == 'Random':
            return FeatureSelector.random_selection(
                self.X_train, self.X_test, config.N_QUBITS
            )

    def train_epoch(self, qnn, X_train, X_test, y_train, y_test):
        """Train for one epoch"""
        epoch_start = time.time()

        # Training
        n_samples = len(X_train)
        indices = np.random.permutation(n_samples)

        for i in range(0, n_samples, config.BATCH_SIZE):
            batch_indices = indices[i:i + config.BATCH_SIZE]
            X_batch = X_train[batch_indices]
            y_batch = y_train[batch_indices]
            qnn.train_step(X_batch, y_batch, config.LEARNING_RATE)

        # Evaluation
        train_pred = qnn.predict_batch(X_train)
        train_acc = np.mean(train_pred == y_train)

        val_pred = qnn.predict_batch(X_test)
        val_acc = np.mean(val_pred == y_test)

        epoch_time = time.time() - epoch_start

        return train_acc, val_acc, epoch_time

    def run(self):
        """Run full training"""
        print(f"\n{'='*70}")
        print(f"STARTING ADAPTIVE QNN TRAINING")
        print(f"{'='*70}")
        print(f"Configuration: {config.N_QUBITS} qubits, {config.N_LAYERS} layers")
        print(f"Strategy: {config.STRATEGY}")
        print(f"Epochs: {config.N_EPOCHS}")
        print(f"{'='*70}\n")

        for epoch in range(config.N_EPOCHS):
            # Select configuration
            config_name = self.controller.select_configuration()
            feature_method, encoding_method = config_name.split('+')

            # Apply feature selection
            X_train_reduced, X_test_reduced = self.apply_feature_selection(feature_method)

            # Create QNN
            qnn = QuantumNeuralNetwork(
                config.N_QUBITS, config.N_LAYERS, encoding_method
            )

            # Train
            train_acc, val_acc, epoch_time = self.train_epoch(
                qnn, X_train_reduced, X_test_reduced, self.y_train, self.y_test
            )

            # Update controller
            self.controller.update(config_name, val_acc)

            # Track history
            self.training_history['train_acc'].append(train_acc)
            self.training_history['val_acc'].append(val_acc)
            self.training_history['config'].append(config_name)
            self.training_history['epoch_time'].append(epoch_time)

            # Progress
            if config.VERBOSE:
                best_config = self.controller.get_best_configuration()
                best_reward = np.mean(self.controller.rewards[best_config]) if self.controller.rewards[best_config] else 0

                print(f"Epoch {epoch+1:3d}/{config.N_EPOCHS} | "
                      f"Config: {config_name:25s} | "
                      f"Train: {train_acc:.4f} | "
                      f"Val: {val_acc:.4f} | "
                      f"Best: {best_config:20s} ({best_reward:.4f}) | "
                      f"Time: {epoch_time:.2f}s")

        self.total_time = time.time() - self.start_time
        self.generate_report()

        if config.SAVE_RESULTS:
            self.save_results()

        if config.PLOT_RESULTS:
            self.plot_results()

    def generate_report(self):
        """Generate report"""
        print(f"\n{'='*70}")
        print("EXPERIMENT REPORT")
        print(f"{'='*70}")

        print(f"\nTraining Summary:")
        print(f"  Total Epochs: {config.N_EPOCHS}")
        print(f"  Total Time: {self.total_time:.2f}s")
        print(f"  Avg Time/Epoch: {self.total_time/config.N_EPOCHS:.2f}s")

        best_config = self.controller.get_best_configuration()
        best_rewards = self.controller.rewards[best_config]

        print(f"\nBest Configuration:")
        print(f"  Config: {best_config}")
        print(f"  Mean Reward: {np.mean(best_rewards):.4f}")
        print(f"  Std Dev: {np.std(best_rewards):.4f}")
        print(f"  Times Selected: {len(best_rewards)}")

        print(f"\nAll Configurations:")
        print(f"{'Configuration':<25s} {'Mean':<10s} {'Count':<8s} {'Std':<10s}")
        print("-" * 70)

        stats = self.controller.get_statistics()
        for stat in stats:
            print(f"{stat['configuration']:<25s} "
                  f"{stat['mean_reward']:<10.4f} "
                  f"{stat['count']:<8d} "
                  f"{stat['std_reward']:<10.4f}")

        print(f"\n{'='*70}\n")

    def save_results(self):
        """Save results to JSON"""
        results = {
            'config': {
                'n_qubits': config.N_QUBITS,
                'n_layers': config.N_LAYERS,
                'n_epochs': config.N_EPOCHS,
                'strategy': config.STRATEGY,
                'dataset': config.DATASET
            },
            'training_history': self.training_history,
            'controller_stats': self.controller.get_statistics(),
            'total_time': self.total_time
        }

        filename = config.OUTPUT_DIR / f'results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2)

        print(f"✓ Results saved: {filename}")

    def plot_results(self):
        """Generate plots"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Plot 1: Training progress
        ax = axes[0, 0]
        epochs = range(1, len(self.training_history['train_acc']) + 1)
        ax.plot(epochs, self.training_history['train_acc'], label='Train', linewidth=2)
        ax.plot(epochs, self.training_history['val_acc'], label='Validation', linewidth=2)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy')
        ax.set_title('Training Progress')
        ax.legend()
        ax.grid(alpha=0.3)

        # Plot 2: Configuration frequency
        ax = axes[0, 1]
        config_counts = {cfg: self.controller.selection_counts[cfg] for cfg in self.configurations}
        configs = list(config_counts.keys())
        counts = list(config_counts.values())

        y_pos = np.arange(len(configs))
        ax.barh(y_pos, counts)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(configs, fontsize=9)
        ax.set_xlabel('Times Selected')
        ax.set_title('Configuration Selection Frequency')
        ax.grid(axis='x', alpha=0.3)

        # Plot 3: Performance by configuration
        ax = axes[1, 0]
        stats = self.controller.get_statistics()
        configs = [s['configuration'] for s in stats]
        means = [s['mean_reward'] for s in stats]
        stds = [s['std_reward'] for s in stats]

        y_pos = np.arange(len(configs))
        ax.barh(y_pos, means, xerr=stds, capsize=5)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(configs, fontsize=9)
        ax.set_xlabel('Mean Validation Accuracy')
        ax.set_title('Performance by Configuration')
        ax.grid(axis='x', alpha=0.3)

        # Plot 4: Configuration timeline
        ax = axes[1, 1]
        unique_configs = list(set(self.training_history['config']))
        config_to_num = {c: i for i, c in enumerate(unique_configs)}
        config_nums = [config_to_num[c] for c in self.training_history['config']]

        scatter = ax.scatter(epochs, config_nums, c=self.training_history['val_acc'],
                           cmap='viridis', s=50, alpha=0.6)
        ax.set_yticks(range(len(unique_configs)))
        ax.set_yticklabels(unique_configs, fontsize=8)
        ax.set_xlabel('Epoch')
        ax.set_title('Configuration Selection Timeline')
        plt.colorbar(scatter, ax=ax, label='Reward')
        ax.grid(alpha=0.3, axis='x')

        plt.tight_layout()

        filename = config.OUTPUT_DIR / f'plots_{datetime.now().strftime("%Y%m%d_%H%M%S")}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"✓ Plots saved: {filename}")

        plt.show()

print("✓ AdaptiveQNNTrainer defined")

✓ AdaptiveQNNTrainer defined


## Run Experiment

In [22]:
# Create and run trainer
trainer = AdaptiveQNNTrainer()
trainer.run()

Loading dataset...
Dataset: wine
Training: 91, Test: 39
Features: 13 → 10

STARTING ADAPTIVE QNN TRAINING
Configuration: 10 qubits, 2 layers
Strategy: ucb
Epochs: 30

Epoch   1/30 | Config: PCA+angle                 | Train: 0.5604 | Val: 0.5385 | Best: PCA+angle            (0.5385) | Time: 2.97s
Epoch   2/30 | Config: PCA+amplitude             | Train: 0.4176 | Val: 0.4103 | Best: PCA+angle            (0.5385) | Time: 2.94s


KeyboardInterrupt: 

## Download Results

In [ ]:
if IN_COLAB:
    from google.colab import files
    import shutil

    # Zip results
    shutil.make_archive('10_qubits_results', 'zip', config.OUTPUT_DIR)

    print("Downloading results...")
    files.download('10_qubits_results.zip')
    print("✓ Download complete!")
else:
    print("Not running on Colab - results saved locally")